## Architecture Search

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# CIFAR-10 dataset (images and labels)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [7]:
# CNN model (Deep but Narrow)
class DeepButNarrowCNN(nn.Module):
    def __init__(self):
        super(DeepButNarrowCNN, self).__init__()
        self.layer1 = nn.Conv2d(3, 32, 3, padding=1)
        self.layer2 = nn.Conv2d(32, 32, 3, padding=1)
        self.layer3 = nn.Conv2d(32, 64, 3, padding=1)
        self.layer4 = nn.Conv2d(64, 64, 3, padding=1)
        self.layer5 = nn.Conv2d(64, 128, 3, padding=1)
        self.layer6 = nn.Conv2d(128, 128, 3, padding=1)
        self.fc = nn.Linear(128 * 4 * 4, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = DeepButNarrowCNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


0it [00:00, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Epoch [1/10], Step [100/782], Loss: 1.8975
Epoch [1/10], Step [200/782], Loss: 1.1866
Epoch [1/10], Step [300/782], Loss: 1.5947
Epoch [1/10], Step [400/782], Loss: 1.1908
Epoch [1/10], Step [500/782], Loss: 1.2909
Epoch [1/10], Step [600/782], Loss: 1.4519
Epoch [1/10], Step [700/782], Loss: 1.2809
Epoch [2/10], Step [100/782], Loss: 1.3379
Epoch [2/10], Step [200/782], Loss: 0.9116
Epoch [2/10], Step [300/782], Loss: 0.7973
Epoch [2/10], Step [400/782], Loss: 1.0004
Epoch [2/10], Step [500/782], Loss: 0.9603
Epoch [2/10], Step [600/782], Loss: 0.8637
Epoch [2/10], Step [700/782], Loss: 0.7086
Epoch [3/10], Step [100/782], Loss: 1.0356
Epoch [3/10], Step [200/782], Loss: 0.8289
Epoch [3/10], Step [300/782], Loss: 0.8743
Epoch [3/10], Step [400/782], Loss: 0.8731
Epoch [3/10], Step [500/782], Loss: 0.8518
Epoch [3/10], Step [600/782], Loss: 0.7563
Epoch [3/10], Step [700/782], Loss: 0.5964
Epoch [4/10], Step [100/782], Loss: 0.7970
Epo

In [8]:
class DeepCNNNoPoolingNoStriding(nn.Module):
    def __init__(self):
        super(DeepCNNNoPoolingNoStriding, self).__init__()
        self.layer1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.layer3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.layer5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        # Keeping the spatial dimension same throughout, so 32x32 input results in 32x32 output after conv layers
        self.fc = nn.Linear(128 * 32 * 32, 10)  # Large number of input features to the FC layer

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = out.view(out.size(0), -1)  # Flatten the output for the fully connected layer
        out = self.fc(out)
        return out

model_no_pooling_no_striding = DeepCNNNoPoolingNoStriding().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_no_pooling_no_striding.parameters(), lr=learning_rate)

# Train the model_no_pooling_no_striding
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_no_pooling_no_striding(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_no_pooling_no_striding
model_no_pooling_no_striding.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_no_pooling_no_striding(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 1.9773
Epoch [1/10], Step [200/782], Loss: 1.7463
Epoch [1/10], Step [300/782], Loss: 1.8459
Epoch [1/10], Step [400/782], Loss: 1.5274
Epoch [1/10], Step [500/782], Loss: 1.5453
Epoch [1/10], Step [600/782], Loss: 1.5219
Epoch [1/10], Step [700/782], Loss: 1.3501
Epoch [2/10], Step [100/782], Loss: 1.1609
Epoch [2/10], Step [200/782], Loss: 1.2708
Epoch [2/10], Step [300/782], Loss: 1.4547
Epoch [2/10], Step [400/782], Loss: 1.0678
Epoch [2/10], Step [500/782], Loss: 1.1947
Epoch [2/10], Step [600/782], Loss: 1.2415
Epoch [2/10], Step [700/782], Loss: 1.1550
Epoch [3/10], Step [100/782], Loss: 0.9440
Epoch [3/10], Step [200/782], Loss: 1.1037
Epoch [3/10], Step [300/782], Loss: 1.0770
Epoch [3/10], Step [400/782], Loss: 1.1476
Epoch [3/10], Step [500/782], Loss: 1.1093
Epoch [3/10], Step [600/782], Loss: 0.9168
Epoch [3/10], Step [700/782], Loss: 0.8647
Epoch [4/10], Step [100/782], Loss: 0.7568
Epoch [4/10], Step [200/782], Loss: 0.7014
Epoch [4/10

In [12]:
class DeepCNNNoPoolingNoStriding(nn.Module):
    def __init__(self):
        super(DeepCNNNoPoolingNoStriding, self).__init__()
        self.layer1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.layer3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.layer5 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        # Keeping the spatial dimension same throughout, so 32x32 input results in 32x32 output after conv layers
        self.fc = nn.Linear(64 * 32 * 32, 10)  # Large number of input features to the FC layer

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = out.view(out.size(0), -1)  # Flatten the output for the fully connected layer
        out = self.fc(out)
        return out

model_no_pooling_no_striding = DeepCNNNoPoolingNoStriding().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_no_pooling_no_striding.parameters(), lr=learning_rate)

# Train the model_no_pooling_no_striding
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_no_pooling_no_striding(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_no_pooling_no_striding
model_no_pooling_no_striding.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_no_pooling_no_striding(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 2.0367
Epoch [1/10], Step [200/782], Loss: 1.6069
Epoch [1/10], Step [300/782], Loss: 1.4365
Epoch [1/10], Step [400/782], Loss: 1.6237
Epoch [1/10], Step [500/782], Loss: 1.1624
Epoch [1/10], Step [600/782], Loss: 1.4353
Epoch [1/10], Step [700/782], Loss: 1.2655
Epoch [2/10], Step [100/782], Loss: 1.2112
Epoch [2/10], Step [200/782], Loss: 0.9067
Epoch [2/10], Step [300/782], Loss: 1.0452
Epoch [2/10], Step [400/782], Loss: 1.0410
Epoch [2/10], Step [500/782], Loss: 1.0729
Epoch [2/10], Step [600/782], Loss: 0.9605
Epoch [2/10], Step [700/782], Loss: 1.0249
Epoch [3/10], Step [100/782], Loss: 0.9521
Epoch [3/10], Step [200/782], Loss: 0.6517
Epoch [3/10], Step [300/782], Loss: 0.9917
Epoch [3/10], Step [400/782], Loss: 0.8162
Epoch [3/10], Step [500/782], Loss: 0.9734
Epoch [3/10], Step [600/782], Loss: 0.7849
Epoch [3/10], Step [700/782], Loss: 0.7072
Epoch [4/10], Step [100/782], Loss: 0.9017
Epoch [4/10], Step [200/782], Loss: 0.6867
Epoch [4/10

In [4]:
class DeepButNarrowCNNNoPooling(nn.Module):
    def __init__(self):
        super(DeepButNarrowCNNNoPooling, self).__init__()
        self.layer1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.layer2 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1)  # Strided convolution for downsampling
        self.layer3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.layer4 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1)  # Strided convolution for downsampling
        self.layer5 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.layer6 = nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1)  # Strided convolution for downsampling
        self.fc = nn.Linear(128 * 4 * 4, 10)  # Adjusted for the downsampling

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model_no_pooling = DeepButNarrowCNNNoPooling().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_no_pooling.parameters(), lr=learning_rate)

# Train the model_no_pooling
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_no_pooling(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_no_pooling
model_no_pooling.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_no_pooling(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Files already downloaded and verified
Epoch [1/10], Step [100/782], Loss: 1.8552
Epoch [1/10], Step [200/782], Loss: 1.6673
Epoch [1/10], Step [300/782], Loss: 1.6695
Epoch [1/10], Step [400/782], Loss: 1.5474
Epoch [1/10], Step [500/782], Loss: 1.5634
Epoch [1/10], Step [600/782], Loss: 1.3211
Epoch [1/10], Step [700/782], Loss: 1.3915
Epoch [2/10], Step [100/782], Loss: 1.0290
Epoch [2/10], Step [200/782], Loss: 1.3137
Epoch [2/10], Step [300/782], Loss: 1.1727
Epoch [2/10], Step [400/782], Loss: 1.1137
Epoch [2/10], Step [500/782], Loss: 0.9542
Epoch [2/10], Step [600/782], Loss: 1.1167
Epoch [2/10], Step [700/782], Loss: 0.9335
Epoch [3/10], Step [100/782], Loss: 0.8416
Epoch [3/10], Step [200/782], Loss: 0.9012
Epoch [3/10], Step [300/782], Loss: 0.9940
Epoch [3/10], Step [400/782], Loss: 1.1360
Epoch [3/10], Step [500/782], Loss: 0.8969
Epoch [3/10], Step [600/782], Loss: 0.6574
Epoch [3/10], Step [700/782], Loss: 1.0711
Epoch [4/10], Step [100/782], Loss: 0.7665
Epoch [4/10], St

In [5]:
class DeepCNNWithAvgPooling(nn.Module):
    def __init__(self):
        super(DeepCNNWithAvgPooling, self).__init__()
        self.layer1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.avgpool1 = nn.AvgPool2d(2, stride=2)
        self.layer3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.avgpool2 = nn.AvgPool2d(2, stride=2)
        self.layer5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.avgpool3 = nn.AvgPool2d(2, stride=2)
        self.fc = nn.Linear(128 * 4 * 4, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = self.avgpool1(out)
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = self.avgpool2(out)
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = self.avgpool3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model_with_avg_pooling = DeepCNNWithAvgPooling().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_avg_pooling.parameters(), lr=learning_rate)

# Train the model_with_avg_pooling
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_with_avg_pooling(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_with_avg_pooling
model_with_avg_pooling.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_avg_pooling(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 1.8212
Epoch [1/10], Step [200/782], Loss: 1.5978
Epoch [1/10], Step [300/782], Loss: 1.6023
Epoch [1/10], Step [400/782], Loss: 1.3932
Epoch [1/10], Step [500/782], Loss: 1.4105
Epoch [1/10], Step [600/782], Loss: 1.4241
Epoch [1/10], Step [700/782], Loss: 1.3741
Epoch [2/10], Step [100/782], Loss: 1.1217
Epoch [2/10], Step [200/782], Loss: 1.1096
Epoch [2/10], Step [300/782], Loss: 1.0970
Epoch [2/10], Step [400/782], Loss: 1.1788
Epoch [2/10], Step [500/782], Loss: 1.0956
Epoch [2/10], Step [600/782], Loss: 0.9277
Epoch [2/10], Step [700/782], Loss: 1.0410
Epoch [3/10], Step [100/782], Loss: 1.0196
Epoch [3/10], Step [200/782], Loss: 1.1829
Epoch [3/10], Step [300/782], Loss: 1.0102
Epoch [3/10], Step [400/782], Loss: 1.0232
Epoch [3/10], Step [500/782], Loss: 0.8163
Epoch [3/10], Step [600/782], Loss: 0.9078
Epoch [3/10], Step [700/782], Loss: 0.7252
Epoch [4/10], Step [100/782], Loss: 0.7129
Epoch [4/10], Step [200/782], Loss: 0.7875
Epoch [4/10

In [11]:
class DeepCNNWithAvgPoolingConstantChannels(nn.Module):
    def __init__(self):
        super(DeepCNNWithAvgPoolingConstantChannels, self).__init__()
        self.layer1 = nn.Conv2d(3, 256, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.avgpool1 = nn.AvgPool2d(2, stride=2)
        self.layer3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.avgpool2 = nn.AvgPool2d(2, stride=2)
        self.layer5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.avgpool3 = nn.AvgPool2d(2, stride=2)
        self.fc = nn.Linear(256 * 4 * 4, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = self.avgpool1(out)
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = self.avgpool2(out)
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = self.avgpool3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model_with_avg_pooling_constant_channels = DeepCNNWithAvgPoolingConstantChannels().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_avg_pooling_constant_channels.parameters(), lr=learning_rate)

# Train the model_with_avg_pooling_constant_channels
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_with_avg_pooling_constant_channels(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_with_avg_pooling_constant_channels
model_with_avg_pooling_constant_channels.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_avg_pooling_constant_channels(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 1.7938
Epoch [1/10], Step [200/782], Loss: 1.7104
Epoch [1/10], Step [300/782], Loss: 1.4952
Epoch [1/10], Step [400/782], Loss: 1.3783
Epoch [1/10], Step [500/782], Loss: 1.2984
Epoch [1/10], Step [600/782], Loss: 1.3925
Epoch [1/10], Step [700/782], Loss: 1.5976
Epoch [2/10], Step [100/782], Loss: 1.1288
Epoch [2/10], Step [200/782], Loss: 1.0793
Epoch [2/10], Step [300/782], Loss: 1.0554
Epoch [2/10], Step [400/782], Loss: 1.0934
Epoch [2/10], Step [500/782], Loss: 0.7481
Epoch [2/10], Step [600/782], Loss: 0.8244
Epoch [2/10], Step [700/782], Loss: 0.8229
Epoch [3/10], Step [100/782], Loss: 0.6758
Epoch [3/10], Step [200/782], Loss: 0.9416
Epoch [3/10], Step [300/782], Loss: 0.5464
Epoch [3/10], Step [400/782], Loss: 0.6421
Epoch [3/10], Step [500/782], Loss: 0.7798
Epoch [3/10], Step [600/782], Loss: 0.7156
Epoch [3/10], Step [700/782], Loss: 0.7198
Epoch [4/10], Step [100/782], Loss: 0.5683
Epoch [4/10], Step [200/782], Loss: 0.5603
Epoch [4/10

In [7]:
class DeepCNNWithAvgPoolingConstantChannels(nn.Module):
    def __init__(self):
        super(DeepCNNWithAvgPoolingConstantChannels, self).__init__()
        self.layer1 = nn.Conv2d(3, 128, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.avgpool1 = nn.AvgPool2d(2, stride=2)
        self.layer3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.avgpool2 = nn.AvgPool2d(2, stride=2)
        self.layer5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.avgpool3 = nn.AvgPool2d(2, stride=2)
        self.fc = nn.Linear(128 * 4 * 4, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = self.avgpool1(out)
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = self.avgpool2(out)
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = self.avgpool3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model_with_avg_pooling_constant_channels = DeepCNNWithAvgPoolingConstantChannels().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_avg_pooling_constant_channels.parameters(), lr=learning_rate)

# Train the model_with_avg_pooling_constant_channels
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_with_avg_pooling_constant_channels(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_with_avg_pooling_constant_channels
model_with_avg_pooling_constant_channels.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_avg_pooling_constant_channels(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 1.8975
Epoch [1/10], Step [200/782], Loss: 1.7205
Epoch [1/10], Step [300/782], Loss: 1.6065
Epoch [1/10], Step [400/782], Loss: 1.3666
Epoch [1/10], Step [500/782], Loss: 1.3453
Epoch [1/10], Step [600/782], Loss: 1.4118
Epoch [1/10], Step [700/782], Loss: 1.2491
Epoch [2/10], Step [100/782], Loss: 1.0472
Epoch [2/10], Step [200/782], Loss: 1.1361
Epoch [2/10], Step [300/782], Loss: 1.0448
Epoch [2/10], Step [400/782], Loss: 1.0611
Epoch [2/10], Step [500/782], Loss: 0.7402
Epoch [2/10], Step [600/782], Loss: 0.7919
Epoch [2/10], Step [700/782], Loss: 0.9403
Epoch [3/10], Step [100/782], Loss: 0.7273
Epoch [3/10], Step [200/782], Loss: 0.7532
Epoch [3/10], Step [300/782], Loss: 0.6992
Epoch [3/10], Step [400/782], Loss: 0.7046
Epoch [3/10], Step [500/782], Loss: 0.6826
Epoch [3/10], Step [600/782], Loss: 0.6004
Epoch [3/10], Step [700/782], Loss: 0.7906
Epoch [4/10], Step [100/782], Loss: 0.6320
Epoch [4/10], Step [200/782], Loss: 0.8736
Epoch [4/10

In [8]:
class DeepCNNWithAvgPoolingConstantChannels(nn.Module):
    def __init__(self):
        super(DeepCNNWithAvgPoolingConstantChannels, self).__init__()
        self.layer1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.avgpool1 = nn.AvgPool2d(2, stride=2)
        self.layer3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.avgpool2 = nn.AvgPool2d(2, stride=2)
        self.layer5 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.avgpool3 = nn.AvgPool2d(2, stride=2)
        self.fc = nn.Linear(64 * 4 * 4, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = self.avgpool1(out)
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = self.avgpool2(out)
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = self.avgpool3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model_with_avg_pooling_constant_channels = DeepCNNWithAvgPoolingConstantChannels().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_avg_pooling_constant_channels.parameters(), lr=learning_rate)

# Train the model_with_avg_pooling_constant_channels
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_with_avg_pooling_constant_channels(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_with_avg_pooling_constant_channels
model_with_avg_pooling_constant_channels.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_avg_pooling_constant_channels(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 1.5930
Epoch [1/10], Step [200/782], Loss: 1.5423
Epoch [1/10], Step [300/782], Loss: 1.4887
Epoch [1/10], Step [400/782], Loss: 1.3591
Epoch [1/10], Step [500/782], Loss: 1.2652
Epoch [1/10], Step [600/782], Loss: 1.3295
Epoch [1/10], Step [700/782], Loss: 1.1923
Epoch [2/10], Step [100/782], Loss: 1.2057
Epoch [2/10], Step [200/782], Loss: 1.2615
Epoch [2/10], Step [300/782], Loss: 1.1962
Epoch [2/10], Step [400/782], Loss: 0.9189
Epoch [2/10], Step [500/782], Loss: 1.0666
Epoch [2/10], Step [600/782], Loss: 0.9643
Epoch [2/10], Step [700/782], Loss: 1.1659
Epoch [3/10], Step [100/782], Loss: 0.9411
Epoch [3/10], Step [200/782], Loss: 0.8487
Epoch [3/10], Step [300/782], Loss: 1.0320
Epoch [3/10], Step [400/782], Loss: 0.8592
Epoch [3/10], Step [500/782], Loss: 0.8117
Epoch [3/10], Step [600/782], Loss: 0.8880
Epoch [3/10], Step [700/782], Loss: 0.9210
Epoch [4/10], Step [100/782], Loss: 0.6578
Epoch [4/10], Step [200/782], Loss: 1.0378
Epoch [4/10

In [9]:
class DeepCNNWithAvgPoolingConstantChannels(nn.Module):
    def __init__(self):
        super(DeepCNNWithAvgPoolingConstantChannels, self).__init__()
        self.layer1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.avgpool1 = nn.AvgPool2d(2, stride=2)
        self.layer3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.avgpool2 = nn.AvgPool2d(2, stride=2)
        self.layer5 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.avgpool3 = nn.AvgPool2d(2, stride=2)
        self.fc = nn.Linear(32 * 4 * 4, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = self.avgpool1(out)
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = self.avgpool2(out)
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = self.avgpool3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model_with_avg_pooling_constant_channels = DeepCNNWithAvgPoolingConstantChannels().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_avg_pooling_constant_channels.parameters(), lr=learning_rate)

# Train the model_with_avg_pooling_constant_channels
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_with_avg_pooling_constant_channels(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_with_avg_pooling_constant_channels
model_with_avg_pooling_constant_channels.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_avg_pooling_constant_channels(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 2.1611
Epoch [1/10], Step [200/782], Loss: 1.7804
Epoch [1/10], Step [300/782], Loss: 1.5337
Epoch [1/10], Step [400/782], Loss: 1.4270
Epoch [1/10], Step [500/782], Loss: 1.7451
Epoch [1/10], Step [600/782], Loss: 1.5150
Epoch [1/10], Step [700/782], Loss: 1.1501
Epoch [2/10], Step [100/782], Loss: 1.5442
Epoch [2/10], Step [200/782], Loss: 1.3839
Epoch [2/10], Step [300/782], Loss: 1.3410
Epoch [2/10], Step [400/782], Loss: 1.3856
Epoch [2/10], Step [500/782], Loss: 1.2276
Epoch [2/10], Step [600/782], Loss: 1.4093
Epoch [2/10], Step [700/782], Loss: 1.2505
Epoch [3/10], Step [100/782], Loss: 1.1723
Epoch [3/10], Step [200/782], Loss: 1.0134
Epoch [3/10], Step [300/782], Loss: 1.3111
Epoch [3/10], Step [400/782], Loss: 0.8358
Epoch [3/10], Step [500/782], Loss: 1.3744
Epoch [3/10], Step [600/782], Loss: 0.9760
Epoch [3/10], Step [700/782], Loss: 0.9385
Epoch [4/10], Step [100/782], Loss: 1.0020
Epoch [4/10], Step [200/782], Loss: 0.8384
Epoch [4/10

In [10]:
class DeepCNNWithAvgPoolingConstantChannels(nn.Module):
    def __init__(self):
        super(DeepCNNWithAvgPoolingConstantChannels, self).__init__()
        self.layer1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.layer2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.avgpool1 = nn.AvgPool2d(2, stride=2)
        self.layer3 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.avgpool2 = nn.AvgPool2d(2, stride=2)
        self.layer5 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.layer6 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.avgpool3 = nn.AvgPool2d(2, stride=2)
        self.fc = nn.Linear(16 * 4 * 4, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = self.avgpool1(out)
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = self.avgpool2(out)
        out = F.relu(self.layer5(out))
        out = F.relu(self.layer6(out))
        out = self.avgpool3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model_with_avg_pooling_constant_channels = DeepCNNWithAvgPoolingConstantChannels().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_avg_pooling_constant_channels.parameters(), lr=learning_rate)

# Train the model_with_avg_pooling_constant_channels
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_with_avg_pooling_constant_channels(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model_with_avg_pooling_constant_channels
model_with_avg_pooling_constant_channels.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_avg_pooling_constant_channels(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Epoch [1/10], Step [100/782], Loss: 2.1601
Epoch [1/10], Step [200/782], Loss: 1.7929
Epoch [1/10], Step [300/782], Loss: 1.8409
Epoch [1/10], Step [400/782], Loss: 1.5978
Epoch [1/10], Step [500/782], Loss: 1.7161
Epoch [1/10], Step [600/782], Loss: 1.5679
Epoch [1/10], Step [700/782], Loss: 1.5398
Epoch [2/10], Step [100/782], Loss: 1.5118
Epoch [2/10], Step [200/782], Loss: 1.4132
Epoch [2/10], Step [300/782], Loss: 1.5023
Epoch [2/10], Step [400/782], Loss: 1.4884
Epoch [2/10], Step [500/782], Loss: 1.3626
Epoch [2/10], Step [600/782], Loss: 1.6282
Epoch [2/10], Step [700/782], Loss: 1.4192
Epoch [3/10], Step [100/782], Loss: 1.4644
Epoch [3/10], Step [200/782], Loss: 1.4361
Epoch [3/10], Step [300/782], Loss: 1.5522
Epoch [3/10], Step [400/782], Loss: 1.3692
Epoch [3/10], Step [500/782], Loss: 1.2385
Epoch [3/10], Step [600/782], Loss: 1.3618
Epoch [3/10], Step [700/782], Loss: 1.6945
Epoch [4/10], Step [100/782], Loss: 1.3187
Epoch [4/10], Step [200/782], Loss: 1.2664
Epoch [4/10